In [1]:
# importing necessary libraries
import tensorflow as tf
from tensorflow import keras
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2


In [19]:
# reading images
class ReadData():
    def __init__(self, path: str):
        self.path = path

    def read(self):
        images = []
        for dirname, _, filenames in os.walk(self.path):
            for filename in filenames:
                pathname = os.path.join(dirname, filename)
                image = Image.open(pathname)


        image = image.resize((224, 224))
        images.append(np.array(image, dtype = np.int64))

        return images

alpacaData = ReadData('/dataset/alpaca')

notAlpacaData = ReadData('/dataset/not alpaca')

alpacaImages = alpacaData.read()
notAlpacaImages = notAlpacaData.read()

UnboundLocalError: local variable 'image' referenced before assignment

In [17]:
alpacaImages

[]

In [11]:
from keras.backend import normalize_batch_in_training
# preprocessing images
class PreprocessImages():
    def __init__(self):
        pass

    def normaliseImages(self, images):
        normalisedImages = []
        for image in images:
            if len(image.shape) != 2:
                normalisedImages.append(image/255)
        return normalisedImages

"""
  def rgbToGray(self, images):
    gray_images = []

    for image in images:
      gray_images.append(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))
    return gray_images

"""
preprocessImages = PreprocessImages()
normalisedAlpacas = preprocessImages.normaliseImages(alpacaImages)
normalisedNotAlpacas = preprocessImages.normaliseImages(notAlpacaImages)
#grayImagesAlpacas = preprocessImages.rgbToGray(normalisedAlpacas)
#grayImagesNotAlpacas = preprocessImages.rgbToGray(normalisedNotAlpacas)

In [12]:
# combining the two class images into one dataset
class CreateDataset():
    def __self__(self):
        pass

    def createDataset(self, trueImages, falseImages):
        dataset = pd.DataFrame(columns = ['images', 'label'])

        for image in trueImages:
            dataset.loc[len(dataset), 'images'] = image
    
        for image in falseImages:
            dataset.loc[len(dataset), 'images'] = image

        dataset.loc[:len(trueImages)-1, 'label'] = 1
        dataset.loc[len(trueImages):, 'label'] = 0
        return dataset

createDataset = CreateDataset()
dataset = createDataset.createDataset(normalisedAlpacas, normalisedNotAlpacas)
print(dataset)

Empty DataFrame
Columns: [images, label]
Index: []


C:\Users\Smitesh\AppData\Local\Temp/ipykernel_2992/3997968691.py:15: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  dataset.loc[:len(trueImages)-1, 'label'] = 1
C:\Users\Smitesh\AppData\Local\Temp/ipykernel_2992/3997968691.py:16: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  dataset.loc[len(trueImages):, 'label'] = 0


In [5]:
# splitting the dataset into training, validation, and testing
trainingData, validationData = train_test_split(dataset,
                                             test_size = 0.3,
                                             random_state = 110323,
                                             stratify = dataset.label)

validationData, testingData = train_test_split(validationData,
                                             test_size = 0.5,
                                             random_state = 110323,
                                             stratify = validationData.label)

trainingData = trainingData.reset_index().drop(columns=['index'])
validationData = validationData.reset_index().drop(columns=['index'])
testingData = testingData.reset_index().drop(columns=['index'])
print(trainingData)

KeyboardInterrupt: 

In [ ]:
class Involution(keras.layers.Layer):
  def __init__(self, channel, group_number, kernel_size, stride, reduction_ratio, name):
      super().__init__(name=name)

      # Initialize the parameters.
      self.channel = channel
      self.group_number = group_number
      self.kernel_size = kernel_size
      self.stride = stride
      self.reduction_ratio = reduction_ratio

  def build(self, input_shape):
      # Get the shape of the input.
      (_, height, width, num_channels) = input_shape

      # Scale the height and width with respect to the strides.
      height = height // self.stride
      width = width // self.stride

      # Define a layer that average pools the input tensor
      # if stride is more than 1.
      self.stride_layer = (
          keras.layers.AveragePooling2D(
              pool_size=self.stride, strides=self.stride, padding="same"
          )
          if self.stride > 1
          else tf.identity
      )
      # Define the kernel generation layer.
      self.kernel_gen = keras.Sequential(
          [
              keras.layers.Conv2D(
                  filters=self.channel // self.reduction_ratio, kernel_size=1
              ),
              keras.layers.BatchNormalization(),
              keras.layers.ReLU(),
              keras.layers.Conv2D(
                  filters=self.kernel_size * self.kernel_size * self.group_number,
                  kernel_size=1,
              ),
          ]
      )
      # Define reshape layers
      self.kernel_reshape = keras.layers.Reshape(
          target_shape=(
              height,
              width,
              self.kernel_size * self.kernel_size,
              1,
              self.group_number,
          )
      )
      self.input_patches_reshape = keras.layers.Reshape(
          target_shape=(
              height,
              width,
              self.kernel_size * self.kernel_size,
              num_channels // self.group_number,
              self.group_number,
          )
      )
      self.output_reshape = keras.layers.Reshape(
          target_shape=(height, width, num_channels)
      )

  def call(self, x):
      # Generate the kernel with respect to the input tensor.
      # B, H, W, K*K*G
      kernel_input = self.stride_layer(x)
      kernel = self.kernel_gen(kernel_input)

      # reshape the kerenl
      # B, H, W, K*K, 1, G
      kernel = self.kernel_reshape(kernel)

      # Extract input patches.
      # B, H, W, K*K*C
      input_patches = tf.image.extract_patches(
          images=x,
          sizes=[1, self.kernel_size, self.kernel_size, 1],
          strides=[1, self.stride, self.stride, 1],
          rates=[1, 1, 1, 1],
          padding="SAME",
      )

      # Reshape the input patches to align with later operations.
      # B, H, W, K*K, C//G, G
      input_patches = self.input_patches_reshape(input_patches)

      # Compute the multiply-add operation of kernels and patches.
      # B, H, W, K*K, C//G, G
      output = tf.multiply(kernel, input_patches)
      # B, H, W, C//G, G
      output = tf.reduce_sum(output, axis=3)

      # Reshape the output kernel.
      # B, H, W, C
      output = self.output_reshape(output)

      # Return the output tensor and the kernel.
      return output, kernel

In [ ]:
X_train = np.array([t for t in trainingData['images']])
y_train = np.array([int(t) for t in trainingData['label']])
X_val = np.array([t for t in validationData['images']])
y_val = np.array([int(t) for t in validationData['label']])
X_test = np.array([t for t in testingData['images']])
y_test = np.array([int(t) for t in testingData['label']])

assert(len(X_train) == len(y_train))
assert(len(X_val) == len(y_val))
assert(len(X_test) == len(y_test))

In [ ]:
from keras.metrics import accuracy

# Build the involution model.
print("building the involution model...")

inputs = keras.Input(shape=(224, 224, 3))
x, _ = Involution(channel=3, group_number=1, kernel_size=3, stride=1, reduction_ratio=2, name="inv_1")(inputs)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPooling2D((2, 2))(x)
x, _ = Involution(channel=3, group_number=1, kernel_size=3, stride=1, reduction_ratio=2, name="inv_2")(x)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPooling2D((2, 2))(x)
x, _ = Involution(channel=3, group_number=1, kernel_size=3, stride=1, reduction_ratio=2, name="inv_3")(x)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPooling2D((2, 2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(10, activation="relu")(x)



outputs = keras.layers.Dense(1, activation="sigmoid")(x)

inv_model = keras.Model(inputs=[inputs], outputs=[outputs], name="inv_model")

inv_model.compile(optimizer =  keras.optimizers.Adam(0.0001),
                  loss='binary_crossentropy',
                  metrics = ["accuracy"])
                  

In [ ]:
inv_model.summary()

In [ ]:
inv_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 300)

In [ ]:
from sklearn.metrics import accuracy_score

preds = inv_model.predict(X_test)
accuracy_score(y_test, np.round(preds))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

cnn = Sequential()

cnn.add(Conv2D(16, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Conv2D(12, kernel_size=(3,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Conv2D(8, kernel_size=(3,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(256, activation='relu'))
cnn.add(Dense(1))
cnn.add(Activation(activation='sigmoid'))

# Compile the model
cnn.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])

cnn.summary()

In [ ]:
cnn.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 200)

In [ ]:
from sklearn.metrics import accuracy_score

preds = cnn.predict(X_test)
accuracy_score(y_test, np.round(preds))

In [22]:
pip install --upgrade jupyter_http_over_ws>=0.0.7 && jupyter serverextension enable --py jupyter_http_over_ws

Note: you may need to restart the kernel to use updated packages.


Enabling: jupyter_http_over_ws
- Writing config: C:\Users\Smitesh\.jupyter
    - Validating...
      jupyter_http_over_ws 0.0.7 ok


In [23]:
jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0

SyntaxError: invalid syntax (Temp/ipykernel_2992/3514443836.py, line 1)